# Recursive Neural Network

### Import libraries and data

In [1]:
import numpy as np
import collections
from llm.config import DATA_DIR, IMAGES_DIR

### Data Preparation

In [2]:
from llm.core.rnn import prepare_data
from llm.core.functions import read_text_file

filepath = DATA_DIR.joinpath('Prometheus.txt')
chap_one = read_text_file(filepath)
X, y, vocab_size, max_length, tokenizer = prepare_data(chap_one)

2024-05-09 18:19:08.198581: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-09 18:19:09.148056: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/marco/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/marco/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/marco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Create & Train the RRN Model

In [4]:
from llm.core.rnn import create_rnn_model

model = create_rnn_model(vocab_size)
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 32ms/step - accuracy: 0.0157 - loss: 4.5462
Epoch 2/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.0797 - loss: 4.3984
Epoch 3/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.0943 - loss: 4.3177
Epoch 4/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.1598 - loss: 4.2303
Epoch 5/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.2557 - loss: 4.1478
Epoch 6/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.3950 - loss: 4.0352
Epoch 7/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.3993 - loss: 3.9617
Epoch 8/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5025 - loss: 3.8557
Epoch 9/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5661 - loss: 3.7591
Epoch 10/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6341 - loss: 3.6454
Epoch 11/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.6926 - loss: 3.5429
Epoch 12/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.7211 - lo

### Generate Text

In [5]:
from llm.core.rnn import generate_text

seed_text = 'The next day, Prometheus'
num_words_to_generate = 20
generated_text = generate_text(model, tokenizer, seed_text, num_words_to_generate, max_length)
generated_text

"The next day, Prometheus and the theft of fire one day zeus distributed gifts to all the gods but he didn't care much for"

In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 135, 50)        │         4,750 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 100)            │        15,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 95)             │         9,595 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 88,337 (345.07 KB)

 Trainable params: 29,445 (115.02 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 58,892 (230.05 KB)

In [7]:
from llm.core.rnn import extract_embeddings
from llm.core.rnn import extract_sentiment_labels
from llm.core.visualization import map_labels_to_colors

embeddings, words = extract_embeddings(model, tokenizer)
category_labels = extract_sentiment_labels(words)
category_labels

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.34,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.4939,
 0.0,
 0.0,
 0.0,
 0.0,
 0.5994,
 0.0,
 -0.0772,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2732,
 0.2263,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.5106,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.2732,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6249,
 0.5574,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [ ]:
from llm.core.visualization import apply_mds
from llm.core.visualization import apply_isomap
from llm.core.visualization import apply_tsne
from llm.core.visualization import plot_embeddings

# Apply dimensionality reduction
embeddings_mds = apply_mds(embeddings)
embeddings_isomap = apply_isomap(embeddings)
embeddings_tsne = apply_tsne(embeddings)

# Plot the results
plot_embeddings(
    embeddings_list=[embeddings_mds, embeddings_isomap, embeddings_tsne],
    words=words,
    titles=['MDS', 'Isomap', 't-SNE'],
    colors=colors
)
